In [1]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, Iterator
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Input
from tensorflow.python.keras import applications
from tensorflow.python.keras.layers import AveragePooling2D, GlobalAveragePooling2D
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, History, TensorBoard
from tensorflow.python.keras.applications.mobilenet import MobileNet
from tensorflow.python.keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras import backend as K
import numpy as np
import pandas as pd
from tensorflow.python.keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.python.keras.models import load_model
from sklearn.preprocessing import OneHotEncoder

In [2]:
#parameters
classes = 12
batch_size=64
train_total = 3820
train_oversampling_total = 6276
validation_total = 930

In [3]:
model_mobilenet = MobileNet(include_top=False, weights = 'imagenet',input_shape=(224,224,3), alpha=0.5)

In [4]:
x = model_mobilenet.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)
model = Model(inputs=model_mobilenet.input, outputs=x)

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 16)      0         
__________

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 20,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [7]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(224,224),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(224,224),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 3820 images belonging to 12 classes.
Found 930 images belonging to 12 classes.


In [8]:
class_indices = validation_generator.class_indices
class_ids = {validation_generator.class_indices[x]: x for x in train_generator.class_indices}

In [9]:
train_gen_temp = []
for i in range(len(class_indices.keys())):
    train_gen_temp.append(train_datagen.flow_from_directory(
                        'train_temp'+'/'+class_ids[i],
                        target_size=(224,224),
                        batch_size=1,
                        class_mode='categorical'))

Found 216 images belonging to 1 classes.
Found 296 images belonging to 1 classes.
Found 222 images belonging to 1 classes.
Found 503 images belonging to 1 classes.
Found 176 images belonging to 1 classes.
Found 386 images belonging to 1 classes.
Found 523 images belonging to 1 classes.
Found 185 images belonging to 1 classes.
Found 409 images belonging to 1 classes.
Found 189 images belonging to 1 classes.
Found 401 images belonging to 1 classes.
Found 314 images belonging to 1 classes.


In [10]:
class train_oversampling_iterator(Iterator):
    def __init__(self,batch_size=32):
        self.batch_size = batch_size
        self.valid_sum = 0
    def next(self):
        image_batch = []
        class_batch = []
        
        for i in range(self.batch_size):
            if(self.valid_sum>=12):
                self.valid_sum = 0
            class_onehot = np.zeros(12)
            image = train_gen_temp[self.valid_sum].next()[0]
            image_batch.append(image[0])
            class_onehot[self.valid_sum] = 1
            class_batch.append(class_onehot)
            self.valid_sum = self.valid_sum + 1
        final_res = (np.array(image_batch),np.array(class_batch))
        return (final_res)

In [11]:
train_oversampling_generator = train_oversampling_iterator(batch_size)

In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-4),
             metrics=['accuracy'])

In [ ]:
checkpointer = ModelCheckpoint(filepath='mobilenet_from_scratch_save_best_alpha_5.hdf5', verbose=1, save_best_only=True, save_weights_only=True)
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=0)
earlystopping = EarlyStopping(monitor='val_loss', patience=20)
history_call = History()
tensorboard = TensorBoard(log_dir='./logs', write_graph=True)

In [ ]:
history = model.fit_generator(
                    train_oversampling_generator,
                    steps_per_epoch = int(np.ceil(train_oversampling_total/batch_size)),
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2,
                    callbacks=[checkpointer, reduceLR, earlystopping, history_call, tensorboard])

In [ ]:
model.load_weights('mobilenet_from_scratch_save_best_alpha_5.hdf5')

In [ ]:
predictions_valid = model.predict_generator(validation_generator,int(np.ceil(validation_total/batch_size)))
predictions_valid = np.argmax(predictions_valid, axis=1)


In [ ]:
true_positive = np.zeros(12)
false_positive = np.zeros(12)
true_negative = np.zeros(12)
false_negative = np.zeros(12)
for i in range(len(predictions_valid)):
    if(predictions_valid[i]==validation_generator.classes[i]):
        true_positive[predictions_valid[i]] = true_positive[predictions_valid[i]] + 1
    else:
        false_positive[predictions_valid[i]] = false_positive[predictions_valid[i]] + 1
        false_negative[validation_generator.classes[i]] = false_negative[validation_generator.classes[i]] +1

In [ ]:
print('true_positive')
print(true_positive)
print('false_positive')
print(false_positive)
print('false_negative')
print(false_negative)
print('Total')
print(true_positive+false_negative)

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(224,224),
                    batch_size=batch_size,
                    class_mode=None)


In [ ]:
predictions = model.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes = [class_ids[x] for x in np.argmax(predictions, axis=1)]


In [ ]:
from os.path import basename
test_ids = [basename(x) for x in test_generator.filenames]
submission = pd.DataFrame({'file':test_ids,'species':predicted_classes})
submission.to_csv('submission_mobilenet_from_scratch.csv', encoding="utf8", index=False)

In [ ]:
from IPython.display import FileLink
FileLink('submission_mobilenet_from_scratch.csv')